# Jupyter Notebook for Vicarious Surgical challenge problem

Author: Joshua Enxing, 1/13/2020.

To run, execute each code cell in sucession. Below is a repasting of the problem description: 

<img src = "Medical Diagnostics.jpg" width = 900 height = 600>

### PDF for Bayesian Networks

The joint probability function for a Bayesian network with nodes $\{X_1,\ldots,X_n\}$ is given by $P(X_1,\ldots,X_n) = \Pi_{i=1}^n P(X_i | \Pi_{X_i})$, where $\Pi_{X_i}$ is the product of all parent nodes of node $X_i$. Hence the joint probability function for this network is: $$P(CP,BP,H,E,D) = P(CP|H)P(BP|H)P(H|E,D)P(E)P(D)$$

In [14]:
#Function that calculates the joint pdf for this Bayesian network
def joint_pdf(cp,bp,e,d,h):
    '''
    joint_pdf(cp,bp,e,d,hd) outputs the probability of the joint
    pdf for the Bayesian network given values cp,bp,hd,e,and d.
    
    cp = 0 or 1 corresponds to no chest pain / chest pain respectively
    bp = 0 or 1 corresponds to low bp / high bp respectively
    e = 0 or 1 corresponds to no exercise / exercise respectively
    d = 0 is an unhealthy diet, d = 1 is a healthy diet
    h = 0 or 1 corresponds to hd / no hd respectively
    '''
    ex = [0.3,0.7]
    diet = [0.75,0.25]
    
    # First dimension is chest pain, second is heart disease
    chest = [[0.99,0.2],[0.01,0.8]]
    
    # First dimension is blood pressure, second is heart disease
    blood = [[0.8,0.15],[0.2,0.85]]
    
    # First dimension is heart disease, second is exercise, third
    # is diet
    heart = [[[0.25,0.55],[0.45,0.75]],[[0.75,0.45],[0.55,0.25]]]
    
    return chest[cp][h]*blood[bp][h]*heart[h][e][d]*ex[e]*diet[d]

### Calculating Probability of Heart Disease

Now we use the pdf to calculate the probability of having heart disease. Let's say we'd like to know the probability of having heart disease given high blood pressure and chest pain. This would be:

$$P(H=T|CP=T,BP=T) = \frac{P(H=T,CP=T,BP=T)}{P(CP=T,BP=T)} = \frac{\Sigma_{E,D\in \{T,F\}}P(CP=T,BP=T,H=T,E,D)}{\Sigma_{E,D,H\in \{T,F\}}P(CP=T,BP=T,H,E,D)}$$

In [15]:
import itertools

# Convert input to 1 for positive, 0 for negative, -1 for not given
def str_to_val(str):
    if str == 'y':
        val = 1
    elif str == 'n':
        val = 0
    else:
        val = -1
    return val
        
# Calculates probability of heart disease given supplied features
def calc_prob_hd(cp,bp,e,d):
    vals = [cp,bp,e,d]
    vals = [str_to_val(i) for i in vals]
    
    # Initialize variables for sums
    n = 0
    den = 0
    
    # Find which features not given
    ind = [i for i, x in enumerate(vals) if x == -1]
    
    # Makes list of 2^(len(ind)) elements, contains all combinations of 
    # zeros and ones for features not supplied as input
    lst = [list(i) for i in itertools.product([0, 1], repeat=len(ind))]
    
    # Go through all combinations of features not supplied as input
    for l in lst:
        # Change feature values that weren't supplied as input
        for i in ind:
            vals[i] = l[ind.index(i)]
        n = n + joint_pdf(vals[0],vals[1],vals[2],vals[3],1) # Values in numerator of probability
        den = den + joint_pdf(vals[0],vals[1],vals[2],vals[3],0) # Values only in denom of probability
    
    # Probability is sum of pdf over all combinations of all features not supplied as input (while h=1),
    # divided by sum of pdf over all combinations of all features not supplied as input and all values of h
    return n/(n+den) 

When enter next code cell requiring user input, input **y** or **n** to say a feature is true or false, and **na** to not use that feature at all. For example, to calculate the probability of heart disease given only that high blood pressure is true and chest pain is true, enter **y** for chest pain and blood pressure questions, and **na** for exercise and healthy diet questions. As another example, to calculate the probability of heart disease for all of the patients, enter **na** for all conditions. The resulting output will be the probability of heart disease for the given conditions.

In [18]:
cp = input("Chest pain? (y/n/na) ").lower()
bp = input("Blood pressure high? (y/n/na) ").lower()
e = input("Exercise? (y/n/na) ").lower()
d = input("Healthy diet? (y/n/na) ").lower()
print("Probability of heart disease is:",calc_prob_hd(cp,bp,e,d))

Chest pain? (y/n/na) N
Blood pressure high? (y/n/na) Y
Exercise? (y/n/na) Y
Healthy diet? (y/n/na) Y
Probability of heart disease is: 0.22251308900523556
